<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/VERTEXAI_DEMO_DEC2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-env -q
import colab_env

## TRAINING

In [ ]:
import os
import time
from google.colab import auth
from google.cloud import aiplatform
from vertexai.preview.tuning import sft
import vertexai
from google.auth import default
from google.auth.transport.requests import Request as AuthRequest
import sys

# --- CRITICAL STEP: MANUAL AUTHENTICATION (Must run successfully) ---
print("--- Interactive Authentication ---")
auth.authenticate_user()

# --- 0. HARDCODED CONFIGURATION (TO BYPASS ENV ERRORS) ---
CONFIG = {
    # ⚠️ HARDCODED CRITICAL PROJECT VALUES (Verified from previous logs) ⚠️
    "PROJECT_ID": "g",
    "PROJECT_NUMBER": ",
    "REGION": "us-central1",
    "BUCKET_NAME": "",

    # Model and Tuning Parameters
    "BASE_MODEL": "gemini-2.0-flash-001",
    "FINAL_MODEL_DISPLAY_NAME": "cmapss-rul-gemini-final-launch-v2", # For reference only
    "EPOCHS": 10,
    "LEARNING_RATE_MULTIPLIER": 1.0,

    # Dataset Files
    "TRAIN_FILE_NAME": "cmapss_FD004_train_text.jsonl",
    "VALIDATION_FILE_NAME": "cmapss_FD004_test_text.jsonl",
}

# Derived URIs
STAGING_BUCKET = f"gs://{CONFIG['BUCKET_NAME']}/staging"
TRAIN_DATASET_URI = f"gs://{CONFIG['BUCKET_NAME']}/{CONFIG['TRAIN_FILE_NAME']}"
VALIDATION_DATASET_URI = f"gs://{CONFIG['BUCKET_NAME']}/{CONFIG['VALIDATION_FILE_NAME']}"


# --- 1. INITIALIZATION (TOKEN REFRESH) ---
print("--- 1. Initializing SDK ---")
try:
    # Refresh credentials using the token established by auth.authenticate_user()
    credentials, project = default()
    credentials.refresh(AuthRequest())

    aiplatform.init(project=CONFIG['PROJECT_ID'], location=CONFIG['REGION'], staging_bucket=STAGING_BUCKET)
    vertexai.init(project=CONFIG['PROJECT_ID'], location=CONFIG['REGION'], staging_bucket=STAGING_BUCKET)
    print(f"✅ Vertex AI SDK initialized for Project: {CONFIG['PROJECT_ID']}")
except Exception as e:
    print(f"❌ Initialization failed: {e}. Please ensure your authentication completed successfully.")
    sys.exit(1)


# --- 2. START THE FINE-TUNING JOB (FINAL SYNTAX) ---
print("\n--- 2. Starting Fine-Tuning Job ---")
try:
    sft_tuning_job = sft.train(
        source_model=CONFIG['BASE_MODEL'],
        train_dataset=TRAIN_DATASET_URI,
        validation_dataset=VALIDATION_DATASET_URI,

        # CRITICAL FIX: The display_name argument is REMOVED entirely to avoid syntax conflict.

        epochs=CONFIG['EPOCHS'],
        learning_rate_multiplier=CONFIG['LEARNING_RATE_MULTIPLIER'],
    )

    # Building a stable URL since SDK attribute was problematic
    job_monitor_url = (f"https://console.cloud.google.com/vertex-ai/locations/{CONFIG['REGION']}/"
                       f"training/{sft_tuning_job.resource_name.split('/')[-1]}?project={CONFIG['PROJECT_ID']}")

    print(f"\n✅ Tuning Job Submitted!")
    print(f"Monitor Job Here: {job_monitor_url}")
    print(f"Job Resource Name (SAVE THIS): {sft_tuning_job.resource_name}")

except Exception as e:
    print(f"\n❌ Job Submission Failed: {e}")

--- Interactive Authentication ---
--- 1. Initializing SDK ---


INFO:vertexai.tuning._tuning:Creating SupervisedTuningJob


✅ Vertex AI SDK initialized for Project: gen-lang-client-0870511801

--- 2. Starting Fine-Tuning Job ---


INFO:vertexai.tuning._tuning:SupervisedTuningJob created. Resource name: projects/677155171887/locations/us-central1/tuningJobs/8849670351223783424
INFO:vertexai.tuning._tuning:To use this SupervisedTuningJob in another session:
INFO:vertexai.tuning._tuning:tuning_job = sft.SupervisedTuningJob('projects/677155171887/locations/us-central1/tuningJobs/8849670351223783424')
INFO:vertexai.tuning._tuning:View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/8849670351223783424?project=677155171887



✅ Tuning Job Submitted!
Monitor Job Here: https://console.cloud.google.com/vertex-ai/locations/us-central1/training/8849670351223783424?project=gen-lang-client-0870511801
Job Resource Name (SAVE THIS): projects/677155171887/locations/us-central1/tuningJobs/8849670351223783424


In [ ]:
import os
import sys
from google.colab import auth
from google.cloud import aiplatform
from vertexai.preview.tuning import sft
import vertexai
from google.auth import default
from google.auth.transport.requests import Request as AuthRequest

# --- CONFIGURATION (HARDCODED IDS) ---
CONFIG = {
    "PROJECT_ID": "XXX",
    "REGION": "us-central1",
    # Target the new, correct job ID
    "TARGET_JOB_ID": "XXXXX",
}

# --- AUTHENTICATION AND INITIALIZATION ---
print("--- 1. Initialization ---")
try:
    auth.authenticate_user()

    credentials, project = default()
    credentials.refresh(AuthRequest())

    aiplatform.init(project=CONFIG['PROJECT_ID'], location=CONFIG['REGION'])
    vertexai.init(project=CONFIG['PROJECT_ID'], location=CONFIG['REGION'])
    print("✅ SDK Initialized.")
except Exception as e:
    print(f"❌ Initialization failed: {e}")
    sys.exit(1)


# --- 2. MONITOR JOB STATUS VIA PYTHON SDK (FIXED) ---
print(f"\n--- 2. Monitoring Job: {CONFIG['TARGET_JOB_ID']} ---")
try:
    # List all tuning jobs in the project
    jobs = sft.SupervisedTuningJob.list()

    found_job = None
    for job in jobs:
        # Check if the resource name ends with the target ID
        if job.resource_name.endswith(CONFIG['TARGET_JOB_ID']):
            found_job = job
            break

    if found_job:
        job_state = found_job.state.name

        # FIX: Removed the conflicting 'found_job.display_name' print statement
        print(f"✅ Job ID: {CONFIG['TARGET_JOB_ID']} found.")
        print(f"Current Job State: {job_state}")

        if job_state == 'JOB_STATE_SUCCEEDED':
            print("\n🎉 JOB SUCCEEDED! You can now run the evaluation cell (Cell 2).")
        elif job_state == 'JOB_STATE_FAILED':
             print("\n❌ JOB FAILED! Please check the logs in the Google Cloud Console.")
        else:
             print("\n⏳ Job is still PENDING or RUNNING. Please wait and re-run this cell.")

    else:
        print(f"❌ Job with ID {CONFIG['TARGET_JOB_ID']} not found in the project list.")

except Exception as e:
    print(f"❌ Monitoring failed: {e}")

--- 1. Initialization ---
✅ SDK Initialized.

--- 2. Monitoring Job: 8849670351223783424 ---
✅ Job ID: 8849670351223783424 found.
Current Job State: JOB_STATE_RUNNING

⏳ Job is still PENDING or RUNNING. Please wait and re-run this cell.


In [ ]:
import os
import sys
import time
from datetime import datetime
from google.colab import auth
from google.cloud import aiplatform
from vertexai.preview.tuning import sft
import vertexai
from google.auth import default
from google.auth.transport.requests import Request as AuthRequest

# --- CONFIGURATION (HARDCODED IDS) ---
CONFIG = {
    "PROJECT_ID": "XX",
    "REGION": "us-central1",
    # Target the correct job ID
    "TARGET_JOB_ID": "",
    "POLLING_INTERVAL_SECONDS": 300, # Check every 5 minutes
}

# --- AUTHENTICATION AND INITIALIZATION ---
print("--- 1. Initialization ---")
try:
    auth.authenticate_user()

    credentials, project = default()
    credentials.refresh(AuthRequest())

    aiplatform.init(project=CONFIG['PROJECT_ID'], location=CONFIG['REGION'])
    vertexai.init(project=CONFIG['PROJECT_ID'], location=CONFIG['REGION'])
    print("✅ SDK Initialized.")
except Exception as e:
    print(f"❌ Initialization failed: {e}")
    sys.exit(1)


# --- 2. MONITOR JOB STATUS WITH WHILE LOOP ---
print(f"\n--- 2. Monitoring Job: {CONFIG['TARGET_JOB_ID']} (Polling every {CONFIG['POLLING_INTERVAL_SECONDS']} seconds) ---")

# Construct the full resource name once
JOB_RESOURCE_NAME = f"projects/{CONFIG['PROJECT_ID']}/locations/{CONFIG['REGION']}/tuningJobs/{CONFIG['TARGET_JOB_ID']}"

try:
    # Get the job object once
    job = sft.SupervisedTuningJob(JOB_RESOURCE_NAME)

    # Get the creation time (it's a datetime object)
    creation_time = job.create_time.replace(tzinfo=None)

    print(f"Job started at: {creation_time.strftime('%Y-%m-%d %H:%M:%S')} UTC")

    while True:
        # Re-instantiate the job object to reliably fetch the absolute latest state and metadata
        job = sft.SupervisedTuningJob(JOB_RESOURCE_NAME)
        current_state = job.state.name

        # Calculate elapsed time
        current_time_utc = datetime.utcnow()
        elapsed_time = current_time_utc - creation_time
        elapsed_minutes = elapsed_time.total_seconds() / 60

        print(f"[{time.strftime('%H:%M:%S')}] State: {current_state} | Elapsed: {elapsed_minutes:.1f} minutes")

        if current_state == 'JOB_STATE_SUCCEEDED':
            print("\n🎉 JOB SUCCEEDED! Exiting monitor loop.")
            break

        elif current_state in ['JOB_STATE_FAILED', 'JOB_STATE_CANCELLED', 'JOB_STATE_ERROR']:
            print(f"\n❌ JOB TERMINATED with state: {current_state}. Exiting monitor loop.")
            break

        # If still running, wait for the defined interval
        print(f"⏳ Waiting {CONFIG['POLLING_INTERVAL_SECONDS']} seconds...")
        time.sleep(CONFIG['POLLING_INTERVAL_SECONDS'])

    print("\n--- Monitoring Complete ---")

except Exception as e:
    print(f"❌ Monitoring loop failed unexpectedly: {e}")

--- 1. Initialization ---
✅ SDK Initialized.

--- 2. Monitoring Job: 8849670351223783424 (Polling every 300 seconds) ---


/usr/local/lib/python3.12/dist-packages/vertexai/tuning/_tuning.py:110: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Job started at: 2025-12-10 06:59:18 UTC


/tmp/ipython-input-1213843493.py:58: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_time_utc = datetime.utcnow()


[07:25:03] State: JOB_STATE_RUNNING | Elapsed: 25.8 minutes
⏳ Waiting 300 seconds...


[07:30:04] State: JOB_STATE_RUNNING | Elapsed: 30.8 minutes
⏳ Waiting 300 seconds...


[07:35:04] State: JOB_STATE_RUNNING | Elapsed: 35.8 minutes
⏳ Waiting 300 seconds...


[07:40:05] State: JOB_STATE_RUNNING | Elapsed: 40.8 minutes
⏳ Waiting 300 seconds...


[07:45:05] State: JOB_STATE_RUNNING | Elapsed: 45.8 minutes
⏳ Waiting 300 seconds...


## EVALUATION

In [ ]:
!pip install rouge-score -q

In [ ]:
import time
import json
import numpy as np
import re
import os
import sys
from google.cloud import aiplatform
from google.colab import auth
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm
from google.auth import default
from google.auth.transport.requests import Request as AuthRequest
from rouge_score import rouge_scorer # Import the ROUGE scorer

# --- IMPORT THE WORKING CLIENTS ---
from google import genai
from google.genai import types
import vertexai

# --- CONFIGURATION (HARDCODED IDS) ---
CONFIG = {
    "PROJECT_ID": "",
    "PROJECT_NUMBER": "",
    "REGION": "us-central1",
    "BUCKET_NAME": "",

    # Prefix to find the latest automatically named endpoint
    "MODEL_DISPLAY_NAME_PREFIX": "tuning-",
    "VALIDATION_FILE_NAME": "cmapss_FD004_test_text.jsonl",
}

# Derived paths
EVAL_DATASET_URI = f"gs://{CONFIG['BUCKET_NAME']}/{CONFIG['VALIDATION_FILE_NAME']}"
LOCAL_DATASET_PATH = '/content/cmapss_FD004_test_text.jsonl'
TUNED_MODEL_ENDPOINT = None # Set dynamically


# --- 1. AUTHENTICATION AND INITIALIZATION ---
print("--- 1. Authentication and Initialization ---")
try:
    auth.authenticate_user()

    # Get the official, robust credentials and refresh the token
    credentials, project = default()
    credentials.refresh(AuthRequest())

    aiplatform.init(project=CONFIG['PROJECT_ID'], location=CONFIG['REGION'])

    # Initialize the working prediction client
    client = genai.Client(
        vertexai=True,
        project=CONFIG['PROJECT_ID'],
        location=CONFIG['REGION'],
    )
    print("✅ Client initialized successfully.")

except Exception as e:
    print(f"❌ Initialization failed: {e}")
    sys.exit(1)


# --- 2. RETRIEVE THE NEW ENDPOINT ID DYNAMICALLY ---
print("\n--- 2. Retrieving New Endpoint ID Dynamically ---")
try:
    # 1. Search for the Endpoint associated with the latest job (which should have the 'tuning-' prefix)
    endpoint_list = aiplatform.Endpoint.list(
        filter=f'display_name:"{CONFIG["MODEL_DISPLAY_NAME_PREFIX"]}"',
        order_by='create_time desc'
    )

    if not endpoint_list:
        raise Exception("Endpoint not found. Please wait for job to finish and deploy.")

    NEW_ENDPOINT_ID = endpoint_list[0].name
    # Construct the final endpoint path using the new ID
    TUNED_MODEL_ENDPOINT = f"projects/{CONFIG['PROJECT_NUMBER']}/locations/{CONFIG['REGION']}/endpoints/{NEW_ENDPOINT_ID}"

    print(f"✅ Found new Endpoint ID: {NEW_ENDPOINT_ID}")
    print(f"✅ Full Endpoint Path: {TUNED_MODEL_ENDPOINT}")

except Exception as e:
    print(f"❌ Failed to find the latest Endpoint: {e}")
    sys.exit(1)


# --- 3. GENERATE AND EVALUATE (RUL + ROUGE METRICS) ---
print("\n--- 3. Starting Prediction and Evaluation ---")
try:
    # Copy the validation dataset locally
    !gsutil cp {EVAL_DATASET_URI} {LOCAL_DATASET_PATH}
    print("✅ Test data copied locally.")

    # Variables for RUL metrics
    ground_truth_ruls = []
    predicted_ruls = []
    ground_truth_texts = [] # For ROUGE
    predicted_texts = []    # For ROUGE
    RUL_PATTERN = re.compile(r'Remaining Useful Life:\s*(\d+\.?\d*)')

    num_lines = sum(1 for line in open(LOCAL_DATASET_PATH))

    with open(LOCAL_DATASET_PATH, 'r') as f:
        for line in tqdm(f, total=num_lines, desc="Running Predictions"):
            data = json.loads(line)

            try:
                prompt = data['contents'][0]['parts'][0]['text']
                gt_text = data['contents'][1]['parts'][0]['text']
            except (IndexError, KeyError):
                continue

            # 1. Extract Ground Truth (GT) RUL & Text
            gt_match = RUL_PATTERN.search(gt_text)
            if gt_match:
                ground_truth_ruls.append(float(gt_match.group(1)))
                ground_truth_texts.append(gt_text)
            else: continue

            # 2. Generate Prediction
            generated_text = ""
            try:
                contents = [prompt]
                for chunk in client.models.generate_content_stream(
                    model=TUNED_MODEL_ENDPOINT,
                    contents=contents,
                    config=types.GenerateContentConfig(
                        temperature=0.0,
                        max_output_tokens=128,
                    ),
                ):
                    generated_text += chunk.text
            except Exception as e:
                print(f"Error during text generation: {e}")
                continue

            # 3. Extract Predicted (P) RUL & Text
            pred_match = RUL_PATTERN.search(generated_text)
            if pred_match:
                predicted_ruls.append(float(pred_match.group(1)))
                predicted_texts.append(generated_text)
            else:
                predicted_ruls.append(0.0)
                predicted_texts.append("") # Append empty string if no RUL found

    # --- 4. CALCULATE REGRESSION METRICS ---
    min_len = min(len(ground_truth_ruls), len(predicted_ruls))
    gt_ruls = np.array(ground_truth_ruls[:min_len])
    pred_ruls = np.array(predicted_ruls[:min_len])

    if min_len > 0:
        mae = mean_absolute_error(gt_ruls, pred_ruls)
        rmse = np.sqrt(mean_squared_error(gt_ruls, pred_ruls))

        print("\n\n--- 4a. RUL Regression Performance ---")
        print(f"Total Test Samples Evaluated: {min_len}")
        print(f"Mean Absolute Error (MAE): {mae:.3f} cycles")
        print(f"Root Mean Squared Error (RMSE): {rmse:.3f} cycles")


    # --- 4. CALCULATE ROUGE METRICS ---
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    total_rouge1_f = 0
    total_rouge2_f = 0
    total_rougeL_f = 0

    # Use the texts collected during prediction
    for ref, hyp in zip(ground_truth_texts, predicted_texts):
        if hyp: # Only calculate ROUGE if a prediction text exists
            scores = scorer.score(ref, hyp)
            total_rouge1_f += scores['rouge1'].fmeasure
            total_rouge2_f += scores['rouge2'].fmeasure
            total_rougeL_f += scores['rougeL'].fmeasure

    num_samples_with_pred = len(predicted_texts)

    if num_samples_with_pred > 0:
        avg_rouge1_f = total_rouge1_f / num_samples_with_pred
        avg_rouge2_f = total_rouge2_f / num_samples_with_pred
        avg_rougeL_f = total_rougeL_f / num_samples_with_pred

        print("\n--- 4b. ROUGE Text Quality Performance ---")
        print(f"Average ROUGE-1 F1 (Word Overlap): {avg_rouge1_f:.4f}")
        print(f"Average ROUGE-2 F1 (Bigram Overlap): {avg_rouge2_f:.4f}")
        print(f"Average ROUGE-L F1 (Longest Common Subsequence): {avg_rougeL_f:.4f}")
        print("Higher ROUGE scores indicate better textual alignment with the ground truth.")
    else:
        print("Evaluation failed: No valid RUL predictions were generated.")

except Exception as e:
    print(f"\n❌ Final Prediction/Evaluation Failed: {e}")
    sys.exit(1)

## DELETED ENDPOINTS

In [ ]:
import time
import sys
from google.cloud import aiplatform
from google.colab import auth

# --- CONFIGURATION (HARDCODED IDS) ---
CONFIG = {
    "PROJECT_ID": "",
    "PROJECT_NUMBER": "",
    "REGION": "us-central1",
    # Prefix used to find the automatically named endpoint
    "MODEL_DISPLAY_NAME_PREFIX": "tuning-",
}

# --- AUTHENTICATION AND INITIALIZATION ---
print("--- 1. Authenticating and Initializing SDK ---")
try:
    auth.authenticate_user()
    aiplatform.init(project=CONFIG['PROJECT_ID'], location=CONFIG['REGION'])
    print("✅ SDK Initialized.")
except Exception as e:
    print(f"❌ Initialization failed: {e}")
    sys.exit(1)


# --- 2. DYNAMICALLY FIND AND DELETE ENDPOINT ---
print("\n--- 2. Finding and Deleting Latest Endpoint ---")
try:
    # 2a. DYNAMICALLY FIND THE LATEST ENDPOINT
    print("Searching for the most recently deployed endpoint...")
    endpoint_list = aiplatform.Endpoint.list(
        filter=f'display_name:"{CONFIG["MODEL_DISPLAY_NAME_PREFIX"]}"',
        order_by='create_time desc'
    )

    if not endpoint_list:
        print("❌ No deployed endpoints found with the 'tuning-' prefix. Cleanup complete.")
        sys.exit(0)

    latest_endpoint = endpoint_list[0]
    ENDPOINT_ID = latest_endpoint.name
    ENDPOINT_RESOURCE_NAME = latest_endpoint.resource_name

    print(f"✅ Found Endpoint ID: {ENDPOINT_ID}")

    # 2b. DELETE THE ENDPOINT
    print("Submitting deletion request...")

    # We use the endpoint object directly for deletion
    delete_operation = latest_endpoint.delete(force=True, sync=False)

    # Wait briefly for the server to acknowledge the request
    time.sleep(5)

    print("\n✅ DELETION SUBMITTED SUCCESSFULLY!")
    print(f"Billing for Endpoint {ENDPOINT_ID} has been stopped.")

except Exception as e:
    print(f"\n❌ Deletion failed. You may need to delete it manually via the Google Cloud Console.")
    print(f"Error: {e}")

## old eval

In [ ]:
import time
import json
import numpy as np
import re
import os
from google.cloud import aiplatform
from google.colab import auth
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm
from google.auth import default
from google.auth.transport.requests import Request as AuthRequest
import sys

# --- IMPORT THE WORKING CLIENTS ---
from google import genai
from google.genai import types
import vertexai

# --- CONFIGURATION (HARDCODED IDS) ---
CONFIG = {
    "PROJECT_ID": "",
    "PROJECT_NUMBER": "",
    "REGION": "us-central1",
    "BUCKET_NAME": "",

    # Prefix to find the latest automatically named endpoint
    "MODEL_DISPLAY_NAME_PREFIX": "tuning-",
    "VALIDATION_FILE_NAME": "cmapss_FD004_test_text.jsonl",
}

# Derived paths
EVAL_DATASET_URI = f"gs://{CONFIG['BUCKET_NAME']}/{CONFIG['VALIDATION_FILE_NAME']}"
LOCAL_DATASET_PATH = '/content/cmapss_FD004_test_text.jsonl'
TUNED_MODEL_ENDPOINT = None # Set dynamically


# --- 1. AUTHENTICATION AND INITIALIZATION ---
print("--- 1. Authentication and Initialization ---")
try:
    auth.authenticate_user()

    # Get the official, robust credentials and refresh the token
    credentials, project = default()
    credentials.refresh(AuthRequest())

    aiplatform.init(project=CONFIG['PROJECT_ID'], location=CONFIG['REGION'])

    # Initialize the working prediction client
    client = genai.Client(
        vertexai=True,
        project=CONFIG['PROJECT_ID'],
        location=CONFIG['REGION'],
    )
    print("✅ Client initialized successfully.")

except Exception as e:
    print(f"❌ Initialization failed: {e}")
    sys.exit(1)


# --- 2. RETRIEVE THE NEW ENDPOINT ID DYNAMICALLY ---
print("\n--- 2. Retrieving New Endpoint ID Dynamically ---")
try:
    # 1. Search for the Endpoint associated with the latest job (which should have the 'tuning-' prefix)
    endpoint_list = aiplatform.Endpoint.list(
        filter=f'display_name:"{CONFIG["MODEL_DISPLAY_NAME_PREFIX"]}"',
        order_by='create_time desc'
    )

    if not endpoint_list:
        raise Exception("Endpoint not found. Please wait for job to finish and deploy.")

    NEW_ENDPOINT_ID = endpoint_list[0].name
    # Construct the final endpoint path using the new ID
    TUNED_MODEL_ENDPOINT = f"projects/{CONFIG['PROJECT_NUMBER']}/locations/{CONFIG['REGION']}/endpoints/{NEW_ENDPOINT_ID}"

    print(f"✅ Found new Endpoint ID: {NEW_ENDPOINT_ID}")
    print(f"✅ Full Endpoint Path: {TUNED_MODEL_ENDPOINT}")

except Exception as e:
    print(f"❌ Failed to find the latest Endpoint: {e}")
    sys.exit(1)


# --- 3. GENERATE AND EVALUATE (RUL METRICS) ---
print("\n--- 3. Starting RUL Prediction and Evaluation ---")
try:
    # Copy the validation dataset locally
    !gsutil cp {EVAL_DATASET_URI} {LOCAL_DATASET_PATH}
    print("✅ Test data copied locally.")

    # Variables for RUL metrics
    ground_truth_ruls = []
    predicted_ruls = []
    RUL_PATTERN = re.compile(r'Remaining Useful Life:\s*(\d+\.?\d*)')

    num_lines = sum(1 for line in open(LOCAL_DATASET_PATH))

    with open(LOCAL_DATASET_PATH, 'r') as f:
        for line in tqdm(f, total=num_lines, desc="Running Predictions"):
            data = json.loads(line)

            try:
                prompt = data['contents'][0]['parts'][0]['text']
                gt_text = data['contents'][1]['parts'][0]['text']
            except (IndexError, KeyError):
                continue

            # 1. Extract Ground Truth (GT) RUL
            gt_match = RUL_PATTERN.search(gt_text)
            if gt_match:
                ground_truth_ruls.append(float(gt_match.group(1)))
            else: continue

            # 2. Generate Prediction using the WORKING CLIENT
            generated_text = ""
            try:
                contents = [prompt]

                for chunk in client.models.generate_content_stream(
                    model=TUNED_MODEL_ENDPOINT,
                    contents=contents,
                    config=types.GenerateContentConfig(
                        temperature=0.0,
                        max_output_tokens=128,
                    ),
                ):
                    generated_text += chunk.text
            except Exception as e:
                print(f"Error during text generation: {e}")
                continue

            # 3. Extract Predicted (P) RUL
            pred_match = RUL_PATTERN.search(generated_text)
            if pred_match:
                predicted_ruls.append(float(pred_match.group(1)))
            else:
                predicted_ruls.append(0.0)

    # --- 4. CALCULATE METRICS ---
    min_len = min(len(ground_truth_ruls), len(predicted_ruls))
    gt_ruls = np.array(ground_truth_ruls[:min_len])
    pred_ruls = np.array(predicted_ruls[:min_len])

    if min_len > 0:
        mae = mean_absolute_error(gt_ruls, pred_ruls)
        rmse = np.sqrt(mean_squared_error(gt_ruls, pred_ruls))

        print("\n\n--- RUL Prediction Performance ---")
        print(f"Total Test Samples Evaluated: {min_len}")
        print(f"Mean Absolute Error (MAE): {mae:.3f} cycles")
        print(f"Root Mean Squared Error (RMSE): {rmse:.3f} cycles")
        print("\nLower MAE and RMSE imply higher accuracy of the regression model.")
    else:
        print("Evaluation failed: No valid RUL predictions were generated.")

except Exception as e:
    print(f"\n❌ Final Prediction/Evaluation Failed: {e}")